In [1]:
#
# Este boilerplate es para que Python encuentre el código 
#
%matplotlib inline

from nbhelpers import add_source_path

add_source_path("..")

Vamos a hacer el análisis de la tabla de F0_MEAN... Carguemos la tabla primero

In [3]:
import os
import pylab
import config
import logging
import pandas as pd
from pandasql import sqldf
from analyzers import correlation_analysis 

pysqldf = lambda q: sqldf(q, globals())

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)


feature = "F0_MEAN"
print os.getcwd()
print os.listdir(config.TABLES_DIR)

df = pd.DataFrame.from_csv(os.path.join(config.TABLES_DIR, "%s.csv" % feature))
df['abs_entrainment'] = abs(df['entrainment'])

print df.columns

nice_df = df[['session', 'speaker', 'task', 'entrainment', u'bored_with_game', u'engaged_in_game',
       u'gives_encouragement', u'making_self_clear', u'planning_what_to_say']]

pysqldf("SELECT * from nice_df order by session, speaker, task LIMIT 10")#.to_latex()

/home/marat/projects/tesis/notebooks
['NOISE_TO_HARMONICS_RATIO.csv', 'SYLLABES_COUNT.csv', 'ENG_MEAN.csv', 'ENG_MAX.csv', 'PHONEMES_COUNT.csv', 'F0_MEAN.csv', 'SOUND_VOICED_LOCAL_SHIMMER.csv', 'F0_MAX.csv', 'PHONEMES_AVG.csv', 'VCD2TOT_FRAMES.csv', 'SYLLABES_AVG.csv']
Index([u'task', u'session', u'count', u'speaker', u'tama_mean', u'entrainment',
       u'best_lag', u'hitid', u'session.1', u'task.1', u'speaker.1',
       u'bored_with_game', u'difficult_for_partner_to_speak',
       u'contributes_to_successful_completion', u'engaged_in_game',
       u'gives_encouragement', u'making_self_clear', u'planning_what_to_say',
       u'dislikes_partner', u'abs_entrainment'],
      dtype='object')


,session,speaker,task,entrainment,bored_with_game,engaged_in_game,gives_encouragement,making_self_clear,planning_what_to_say
0,1,0,10,0.581475,0,5,5,5,3
1,1,0,12,-0.569677,1,5,5,5,3
2,1,0,13,0.533701,2,4,5,4,3
3,1,1,10,-0.917101,0,5,2,3,0
4,1,1,12,0.467112,0,5,4,2,4
5,1,1,13,-0.602364,0,5,4,3,1
6,2,0,3,0.520696,0,4,5,5,2
7,2,0,4,-0.241060,0,5,4,4,4
8,2,0,7,0.743719,0,5,4,5,3
9,2,0,8,0.147362,0,5,4,2,2


Haremos lo siguiente:

- Primero agruparemos por sesión, calculando el promedio del entrainment y de cada variable social

In [4]:
avg_by_session = df.groupby(by=["session", "speaker"]).mean()

avg_by_session = avg_by_session[[u'entrainment', u'abs_entrainment',
       u'bored_with_game',
       u'difficult_for_partner_to_speak',
       u'contributes_to_successful_completion', u'engaged_in_game',
       u'gives_encouragement', u'making_self_clear', u'planning_what_to_say',
       u'dislikes_partner']]

avg_by_session

entrainment  abs_entrainment  bored_with_game  \
session speaker                                                  
1       0           0.181833         0.561617         1.000000   
        1          -0.350784         0.662192         0.000000   
2       0           0.078182         0.470979         0.333333   
        1           0.023308         0.287418         2.222222   
3       0           0.132709         0.409746         0.500000   
        1           0.220145         0.458886         1.000000   
4       0           0.055111         0.417121         0.500000   
        1           0.296226         0.380179         0.500000   
5       0           0.114633         0.468439         1.500000   
        1          -0.014086         0.323298         1.000000   
6       0           0.255182         0.551821         0.777778   
        1           0.171634         0.528245         2.111111   
7       0           0.100135         0.482966         0.200000   
        1           0.016517         0.377161         0.600000   
8       0           0.088855         0.520960         0.444444   
        1          -0.150209         0.466073         0.111111   
9       0           0.090275         0.573275         0.600000   
        1           0.090939         0.558944         1.000000   
10      0           0.123270         0.364494         0.428571   
        1           0.063718         0.366301         0.571429   
11      0          -0.310385         0.586370         1.000000   
        1           0.102302         0.624610         0.000000   
12      0           0.189575         0.573661         0.400000   
        1           0.223629         0.547226         0.800000   

                 difficult_for_partner_to_speak  \
session speaker                                   
1       0                              0.666667   
        1                              3.000000   
2       0                              0.333333   
        1                              0.444444   
3       0                              1.166667   
        1                              0.666667   
4       0                              0.500000   
        1                              1.100000   
5       0                              1.000000   
        1                              0.333333   
6       0                              0.333333   
        1                              0.333333   
7       0                              1.300000   
        1                              0.800000   
8       0                              1.222222   
        1                              0.888889   
9       0                              0.400000   
        1                              0.400000   
10      0                              0.571429   
        1                              0.142857   
11      0                              0.555556   
        1                              1.666667   
12      0                              0.500000   
        1                              0.400000   

                 contributes_to_successful_completion  engaged_in_game  \
session speaker                                                          
1       0                                    4.666667         4.666667   
        1                                    3.333333         5.000000   
2       0                                    4.444444         4.777778   
        1                                    3.888889         3.333333   
3       0                                    4.500000         4.666667   
        1                                    4.833333         4.666667   
4       0                                    4.400000         4.300000   
        1                                    5.000000         4.400000   
5       0                                    3.333333         4.166667   
        1                                    4.666667         4.000000   
6       0                                    4.444444         4.444444   
        1  

In [5]:
import social
from analyzers import correlation_analysis

correlation_analysis(avg_by_session)

contributes_to_successful_completion    0.220528
making_self_clear                       0.429411
engaged_in_game                        -0.029591
planning_what_to_say                    0.371073
gives_encouragement                     0.147885
difficult_for_partner_to_speak         -0.426276
bored_with_game                         0.174091
dislikes_partner                       -0.079197
dtype: float64

Ahora hagámoslo pero sobre el valor absoluto del entrainment...

In [6]:
correlation_analysis(avg_by_session, entrainment_var="abs_entrainment")

contributes_to_successful_completion   -0.364292
making_self_clear                      -0.310464
engaged_in_game                         0.292287
planning_what_to_say                   -0.346762
gives_encouragement                     0.357909
difficult_for_partner_to_speak          0.386590
bored_with_game                        -0.276179
dislikes_partner                       -0.229039
dtype: float64

Hagamos una última cosa:
1. Primero, filtremos sólo los entrainments significativos, es decir, aquellas filas tal que $ entrainment > \frac{2}{\sqrt{n}}$

In [7]:
from math import sqrt
from IPython.display import display

significatives = df[abs(df.entrainment)**2 * df['count'] > 4]

avg_by_session = df.groupby(by=["session", "speaker"]).mean()

avg_by_session = avg_by_session[[u'entrainment', u'abs_entrainment',
       u'bored_with_game',
       u'difficult_for_partner_to_speak',
       u'contributes_to_successful_completion', u'engaged_in_game',
       u'gives_encouragement', u'making_self_clear', u'planning_what_to_say',
       u'dislikes_partner']]

correlation_analysis(avg_by_session)


contributes_to_successful_completion    0.220528
making_self_clear                       0.429411
engaged_in_game                        -0.029591
planning_what_to_say                    0.371073
gives_encouragement                     0.147885
difficult_for_partner_to_speak         -0.426276
bored_with_game                         0.174091
dislikes_partner                       -0.079197
dtype: float64

In [ ]:
correlation_analysis(significatives, entrainment_var="abs_entrainment")